# Set project variables

In [ ]:
project_id = "bigquery-demo-282711"
topic_id = "streaming_demo"
topic_path = "projects/bigquery-demo-282711/topics/streaming_demo"
table_id = "bigquery-demo-282711:sales_dataset.sales"
bucket = "bq-demo-dip-realtime-ws"

# Install and import required packages

Use the first command if you usually use pip3 to install packages, or the second one if pip3 has been installed under the alias pip (for example with the Docker image).

In [ ]:
!pip3 install -r ./requirements.txt

In [ ]:
!pip install -r ./requirements.txt

In [ ]:
import datetime
import json
import time
import random

from faker import Faker
from faker.providers import person, address, barcode, date_time
from google.cloud import pubsub_v1

# Create connection with Pub/Sub through service account

In [ ]:
publisher = pubsub_v1.PublisherClient.from_service_account_json("./service_account.json")
topic_path = publisher.topic_path(project_id, topic_id)

# Send fake sales to topic every few seconds

In [ ]:
keep_running = True
fake = Faker()
fake.add_provider(address)
fake.add_provider(person)
fake.add_provider(barcode)
fake.add_provider(date_time)

while keep_running:
    # generate dict with fake sale
    sale = {
        'event_timestamp': datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S"),
        'tr_time_str': fake.date_time_this_month().strftime("%Y-%m-%dT%H:%M:%S"),
        'first_name': fake.first_name(),
        'last_name': fake.last_name(),
        'city': fake.city(),
        'state': str(fake.postcode()),
        'product': str(fake.ean(length=13)),
        'amount': str(random.randrange(1, 5))
    }
    
    # send bytestring representation of dictionary
    publisher.publish(
        topic_path, json.dumps(sale).encode('utf-8')
    )
    
    # wait random number of seconds (between 2 and 7) before sending the next sale
    time.sleep(random.randrange(2, 7))